[doc](https://spark.apache.org/docs/latest/sql-getting-started.html)

# Starting Point: SparkSession
The entry point into all functionality in Spark is the SparkSession class. To create a basic SparkSession, just use SparkSession.builder():<br>
#### 取得SparkSession的物件，SparkSession功能為程式的起點，類比Java的main()

In [1]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession
  .builder()
  .appName("Spark SQL instacart example")
  .config("spark.some.config.option", "some-value")
  .getOrCreate()

spark = org.apache.spark.sql.SparkSession@2401199


org.apache.spark.sql.SparkSession@2401199

In [2]:
spark.version

2.4.4

# Creating DataFrames<br>
#### 讀取資料instacart的資料集(看到後面面程式碼會發現，我只使用products.csv 和 order_products__train.csv來跑關聯性分析

In [3]:
val aisles = spark.read.format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .load("aisles.csv")
val departments = spark.read.format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .load("departments.csv")
val order_products_prior = spark.read.format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .load("order_products__prior.csv")
val order_products_train = spark.read.format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .load("order_products__train.csv")
val orders = spark.read.format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .load("orders.csv")
val products= spark.read.format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .load("products.csv")

aisles = [aisle_id: int, aisle: string]
departments = [department_id: int, department: string]
order_products_prior = [order_id: int, product_id: int ... 2 more fields]
order_products_train = [order_id: int, product_id: int ... 2 more fields]
orders = [order_id: int, user_id: int ... 5 more fields]
products = [product_id: int, product_name: string ... 2 more fields]


[product_id: int, product_name: string ... 2 more fields]

# Running SQL Queries Programmatically
The sql function on a SparkSession enables applications to run SQL queries programmatically and returns the result as a DataFrame.
#### 建立tempview可以使用sql語法進行操作資料(ex: inner join)

In [4]:
// Register the DataFrame as a SQL temporary view
aisles.createOrReplaceTempView("aisles")
departments.createOrReplaceTempView("departments")
order_products_prior.createOrReplaceTempView("order_products_prior")
order_products_train.createOrReplaceTempView("order_products_train")
orders.createOrReplaceTempView("orders")
products.createOrReplaceTempView("products")

In [5]:
// Organize the data by shopping basket
import org.apache.spark.sql.functions.{collect_set,col,count}
//order_products_train只顯示product_id，為了知道product_id的代表什麼商品，與order.csv進行inner join
val rawData = spark.sql("""
select p.product_name, o.order_id 
from products p 
inner join order_products_train o 
where o.product_id = p.product_id""")
//order_products_train的欄位 : order_id, product_id,add_to_cart_order ，但我只想得知每次消費購物籃有哪幾項商品，以order_id groupby
val baskets = rawData.groupBy("order_id").agg(collect_set("product_name").alias("items"))
baskets.createOrReplaceTempView("baskets")
baskets.show(3)

+--------+--------------------+
|order_id|               items|
+--------+--------------------+
|    1342|[Raw Shrimp, Seed...|
|    1591|[Cracked Wheat, S...|
|    4519|[Beet Apple Carro...|
+--------+--------------------+
only showing top 3 rows



rawData = [product_name: string, order_id: int]
baskets = [order_id: int, items: array<string>]


[order_id: int, items: array<string>]

In [6]:
baskets.getClass.getName

org.apache.spark.sql.Dataset

# Train ML Model
[Frequent Pattern Mining - RDD-based API](https://spark.apache.org/docs/latest/mllib-frequent-pattern-mining.html#fp-growth)   文件<br>
[購物籃分析的說明](https://towardsdatascience.com/a-gentle-introduction-on-market-basket-analysis-association-rules-fa4b986a40ce)

In [7]:
// Extract out the items 
//轉成模型可以吃的資料型態
val baskets_ds = spark.sql("select items from baskets").as[Array[String]].toDF("items")

baskets_ds = [items: array<string>]


[items: array<string>]

In [8]:
//看一下data的模樣，每一列代表每次購物的消費項目
baskets_ds.createOrReplaceTempView("watch")
val qwe = spark.sql("select * from watch")
qwe.show(10)

+--------------------+
|               items|
+--------------------+
|[Raw Shrimp, Seed...|
|[Cracked Wheat, S...|
|[Beet Apple Carro...|
|             [Vodka]|
|[Globe Eggplant, ...|
|[Organic Baby Spi...|
|[Reduced Fat Crac...|
|[Organic Red Onio...|
|[Organic Cripps P...|
|[Organic Baby Spi...|
+--------------------+
only showing top 10 rows



qwe = [items: array<string>]


[items: array<string>]

In [9]:
import org.apache.spark.ml.fpm.FPGrowth
// Use FPGrowth
//設定參數(support、confidence)李御璽老師教學時設定是50%，但實際在跑code時，設50%符合的筆數較少
val fpgrowth = new FPGrowth().setItemsCol("items").setMinSupport(0.001).setMinConfidence(0)
val model = fpgrowth.fit(baskets_ds)

fpgrowth = fpgrowth_af81f84fbda6
model = fpgrowth_af81f84fbda6


fpgrowth_af81f84fbda6

## Most Frequent Itemsets

In [10]:
// Display frequent itemsets
val mostPopularItemInABasket = model.freqItemsets
mostPopularItemInABasket.createOrReplaceTempView("mostPopularItemInABasket")

mostPopularItemInABasket = [items: array<string>, freq: bigint]


[items: array<string>, freq: bigint]

In [11]:
//尋找消費者購物的模式，設定購物項目最少2樣商品，並者出頻率最高的消費組合
val sqlDF = spark.sql("select items, freq from mostPopularItemInABasket where size(items) > 2 order by freq desc limit 20")
sqlDF.show()

+--------------------+----+
|               items|freq|
+--------------------+----+
|[Organic Hass Avo...| 710|
|[Organic Raspberr...| 649|
|[Organic Baby Spi...| 587|
|[Organic Raspberr...| 531|
|[Organic Hass Avo...| 497|
|[Organic Avocado,...| 484|
|[Organic Avocado,...| 477|
|[Limes, Large Lem...| 452|
|[Organic Cucumber...| 424|
|[Limes, Organic A...| 389|
|[Organic Raspberr...| 381|
|[Organic Avocado,...| 379|
|[Organic Baby Spi...| 376|
|[Organic Blueberr...| 374|
|[Large Lemon, Org...| 371|
|[Organic Cucumber...| 366|
|[Organic Lemon, O...| 353|
|[Limes, Organic A...| 352|
|[Organic Whole Mi...| 339|
|[Organic Avocado,...| 334|
+--------------------+----+



sqlDF = [items: array<string>, freq: bigint]


[items: array<string>, freq: bigint]

In [12]:
sqlDF.getClass.getName

org.apache.spark.sql.Dataset

# Association Rules
#### 簡單說明:消費者購買褲子，我有多少信心說他會買內褲

## View Generated Association Rules

In [13]:
// Display generated association rules.
val ifThen = model.associationRules
ifThen.createOrReplaceTempView("ifThen")

ifThen = [antecedent: array<string>, consequent: array<string> ... 2 more fields]


[antecedent: array<string>, consequent: array<string> ... 2 more fields]

In [14]:
val sqlDF2 = spark.sql("select antecedent as `antecedent (if)`, consequent as `consequent (then)`, confidence from ifThen order by confidence desc limit 20")
sqlDF2.show()

+--------------------+--------------------+-------------------+
|     antecedent (if)|   consequent (then)|         confidence|
+--------------------+--------------------+-------------------+
|[Organic Raspberr...|[Bag of Organic B...| 0.5984251968503937|
|[Organic Cucumber...|[Bag of Organic B...|           0.546875|
|[Organic Kiwi, Or...|[Bag of Organic B...| 0.5459770114942529|
|[Organic Navel Or...|[Bag of Organic B...| 0.5412186379928315|
|[Yellow Onions, S...|            [Banana]| 0.5357142857142857|
|[Organic Whole St...|[Bag of Organic B...| 0.5314685314685315|
|[Organic Navel Or...|[Bag of Organic B...| 0.5283018867924528|
|[Organic Raspberr...|[Bag of Organic B...|  0.521099116781158|
|[Organic D'Anjou ...|[Bag of Organic B...| 0.5170454545454546|
|[Organic Unsweete...|[Bag of Organic B...| 0.5141065830721003|
|[Organic Broccoli...|[Bag of Organic B...| 0.5048231511254019|
|[Organic Lemon, O...|[Bag of Organic B...| 0.4989106753812636|
|[Organic Hass Avo...|[Bag of Organic B.

sqlDF2 = [antecedent (if): array<string>, consequent (then): array<string> ... 1 more field]


[antecedent (if): array<string>, consequent (then): array<string> ... 1 more field]